In [2]:
#Importation des libraries
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

In [71]:
data_clean = pd.read_csv("data_clean.csv", sep=",")
data = data_clean.drop(columns=['Y_lag_1','Y_rolling_mean_6','Y_rolling_std_6'])

In [72]:
data_filtered=data.copy()

In [61]:
#n'est pas excuter
numeric_data = data.select_dtypes(include=['number'])
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

data_filtered = data[~((numeric_data < (Q1 - 1.5 * IQR)) | 
                       (numeric_data > (Q3 + 1.5 * IQR))).any(axis=1)]

In [73]:
import pandas as pd
from sklearn.decomposition import PCA
# Conversion des colonnes en variables catégorielles
data_filtered['hour'] = data_filtered['hour'].astype('category')
data_filtered['weekday'] = data_filtered['weekday'].astype('category')
data_filtered['month'] = data_filtered['month'].astype('category')
data_filtered['is_weekend'] = data_filtered['is_weekend'].astype('category')
# Étape 1 : Encoder les colonnes catégoriques
data_with_dummies = pd.get_dummies(
    data_filtered, 
    columns=['hour', 'weekday', 'month', 'is_weekend'], 
    drop_first=True
)
# Convertir toutes les colonnes booléennes en entiers
data_with_dummies = data_with_dummies.astype({col: 'int' for col in data_with_dummies.select_dtypes('bool').columns})

In [ ]:
data_with_dummies.info()


In [105]:
# Tri des données pour respecter l'ordre chronologique
datan = data_with_dummies.sort_values(by='DELIVERY_START').reset_index(drop=True)

# Séparation chronologique des ensembles
train_size = 0.6
test_1_size = 0.2

n = len(datan)
train_end = int(n * train_size)
test_1_end = train_end + int(n * test_1_size)
#les variables pour le random
train_f = data_filtered.iloc[:train_end]
test_1_f = data_filtered.iloc[train_end:test_1_end]
test_2_f = data_filtered.iloc[test_1_end:]


train = datan.iloc[:train_end]
test_1 = datan.iloc[train_end:test_1_end]
test_2 = datan.iloc[test_1_end:]

# Définir les variables indépendantes et dépendantes
X_train = train.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_train = train['spot_id_delta']
X_test1 = test_1.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_test1 = test_1['spot_id_delta']
X_test2 = test_2.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_test2 = test_2['spot_id_delta']

# Définir les variables indépendantes et dépendantes
X_train_f = train_f.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_train_f = train_f['spot_id_delta']
X_test1_f = test_1_f.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_test1_f = test_1_f['spot_id_delta']
X_test2_f = test_2_f.drop(columns=['spot_id_delta', 'DELIVERY_START'])
y_test2_f = test_2_f['spot_id_delta']

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test1_scaled = scaler.transform(X_test1)
X_test2_scaled = scaler.transform(X_test2)

# Fonction d'évaluation des métriques avec affichage clair
def evaluate_model(y_true, y_pred, model_name, dataset_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    sign_accuracy = np.mean(np.sign(y_true) == np.sign(y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"=== {model_name} - {dataset_name} ===")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Sign Accuracy: {sign_accuracy:.2f}")
    print(f"R² Score: {r2:.2f}\n")
    return mse, mae, sign_accuracy, r2

# Model
__Random forest__

In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout,LeakyReLU
from tensorflow.keras.optimizers import Adam

In [129]:
# 1. Modèle de Forêt Aléatoire
rf_model = RandomForestRegressor(
    n_estimators=500,  # Augmentation des arbres
    max_depth=20,  # Profondeur ajustée
    min_samples_split=5,  # Réduction du seuil pour splits
    min_samples_leaf=6,  # Feuilles plus petites
    random_state=42,
    bootstrap=True,
)
rf_model.fit(X_train_f, y_train_f)

# Prédictions Forêt Aléatoire
y_pred_rf_train = rf_model.predict(X_train_f)
y_pred_rf_test1 = rf_model.predict(X_test1_f)
y_pred_rf_test2 = rf_model.predict(X_test2_f)

# Évaluation des métriques
# Forêt Aléatoire
evaluate_model(y_train, y_pred_rf_train, "Forêt Aléatoire", "Train")
evaluate_model(y_test1, y_pred_rf_test1, "Forêt Aléatoire", "Test 1")
evaluate_model(y_test2, y_pred_rf_test2, "Forêt Aléatoire", "Test 2")

=== Forêt Aléatoire - Train ===
Mean Squared Error (MSE): 2420.14
Mean Absolute Error (MAE): 26.05
Sign Accuracy: 0.50
R² Score: -0.16

=== Forêt Aléatoire - Test 1 ===
Mean Squared Error (MSE): 1207.39
Mean Absolute Error (MAE): 27.79
Sign Accuracy: 0.51
R² Score: -0.59

=== Forêt Aléatoire - Test 2 ===
Mean Squared Error (MSE): 623.90
Mean Absolute Error (MAE): 18.44
Sign Accuracy: 0.56
R² Score: -1.26



(np.float64(623.8964944532585),
 np.float64(18.44474553391911),
 np.float64(0.5622032288698955),
 -1.2569302619424918)

In [131]:
# Importance des variables pour Random Forest
importances = rf_model.feature_importances_
feature_names = X_train_f.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(importance_df.sort_values(by='Importance', ascending=False))


                          Feature  Importance
3        nucelear_power_available    0.187138
0                   load_forecast    0.160845
8            predicted_spot_price    0.102421
4    wind_power_forecasts_average    0.094392
15                    sin_weekday    0.076014
6        wind_power_forecasts_std    0.062274
10                        weekday    0.054843
7       solar_power_forecasts_std    0.033042
17                      sin_month    0.029349
19                   Y_gas_mean_6    0.027613
9                            hour    0.024393
2             gas_power_available    0.021986
16                    cos_weekday    0.021537
5   solar_power_forecasts_average    0.020671
20                    Y_gas_std_6    0.018823
13                       sin_hour    0.015394
21                  Y_coal_mean_6    0.012167
11                          month    0.011124
1            coal_power_available    0.008897
18                      cos_month    0.006831
14                       cos_hour 

In [143]:
# Supprimer les variables avec une importance proche de 0
threshold = 0.02
low_importance_features = importance_df[importance_df['Importance'] < threshold]['Feature']
X_train_reduced = X_train_f.drop(columns=low_importance_features)
X_test1_reduced = X_test1_f.drop(columns=low_importance_features)
X_test2_reduced = X_test2_f.drop(columns=low_importance_features)

__Apres suppression__

In [134]:
X_train_reduced.shape

(6318, 18)

In [190]:
# 1. Modèle de Forêt Aléatoire
rf_model_f = RandomForestRegressor(
    n_estimators=1000,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    bootstrap=True
)

rf_model_f.fit(X_train_reduced, y_train)

# Prédictions Forêt Aléatoire
y_pred_rf_train_reduced = rf_model_f.predict(X_train_reduced)
y_pred_rf_test1_reduced = rf_model_f.predict(X_test1_reduced)
y_pred_rf_test2_reduced = rf_model_f.predict(X_test2_reduced)

# Évaluation des métriques
# Forêt Aléatoire
evaluate_model(y_train_f, y_pred_rf_train_reduced, "Forêt Aléatoire", "Train")
evaluate_model(y_test1_f, y_pred_rf_test1_reduced, "Forêt Aléatoire", "Test 1")
evaluate_model(y_test2_f, y_pred_rf_test2_reduced, "Forêt Aléatoire", "Test 2")

=== Forêt Aléatoire - Train ===
Mean Squared Error (MSE): 1995.94
Mean Absolute Error (MAE): 22.68
Sign Accuracy: 0.51
R² Score: -0.53

=== Forêt Aléatoire - Test 1 ===
Mean Squared Error (MSE): 1776.33
Mean Absolute Error (MAE): 29.56
Sign Accuracy: 0.48
R² Score: -0.28

=== Forêt Aléatoire - Test 2 ===
Mean Squared Error (MSE): 2151.65
Mean Absolute Error (MAE): 30.15
Sign Accuracy: 0.52
R² Score: -0.07



(np.float64(2151.645309555837),
 np.float64(30.153674293855886),
 np.float64(0.5242165242165242),
 -0.07444750134562317)

__Reseau neurones__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import Huber

# Modèle optimisé
nn_model = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='selu', kernel_regularizer='l2'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='selu', kernel_regularizer='l2'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1)
])

nn_model.compile(optimizer=Adam(learning_rate=0.00005), loss=Huber(delta=1.0))

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Entraînement
nn_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test1_scaled, y_test1),
    epochs=200,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


# Prédictions Réseau de Neurones
y_pred_nn_train = nn_model.predict(X_train_scaled).flatten()
y_pred_nn_test1 = nn_model.predict(X_test1_scaled).flatten()
y_pred_nn_test2 = nn_model.predict(X_test2_scaled).flatten()


# Réseau de Neurones
evaluate_model(y_train, y_pred_nn_train, "Réseau de Neurones", "Train")
evaluate_model(y_test1, y_pred_nn_test1, "Réseau de Neurones", "Test 1")
evaluate_model(y_test2, y_pred_nn_test2, "Réseau de Neurones", "Test 2")

__Boosting__

In [164]:
from sklearn.ensemble import GradientBoostingRegressor

# Modèle Gradient Boosting
gb_model_f = GradientBoostingRegressor(
    n_estimators=700,  # Nombre d'arbres dans le modèle
    learning_rate=0.001,  # Taux d'apprentissage
    #max_depth=20,         # Profondeur maximale des arbres
    min_samples_split=10,  # Nombre minimum d'échantillons pour diviser un noeud
    min_samples_leaf=4,    # Nombre minimum d'échantillons dans une feuille
    random_state=42
)

# Entraînement du modèle
gb_model_f.fit(X_train_f, y_train)

# Prédictions
y_pred_gb_train_reduced = gb_model_f.predict(X_train_f)
y_pred_gb_test1_reduced = gb_model_f.predict(X_test1_f)
y_pred_gb_test2_reduced = gb_model_f.predict(X_test2_f)

# Évaluation des métriques
evaluate_model(y_train_f, y_pred_gb_train_reduced, "Gradient Boosting", "Train")
evaluate_model(y_test1_f, y_pred_gb_test1_reduced, "Gradient Boosting", "Test 1")
evaluate_model(y_test2_f, y_pred_gb_test2_reduced, "Gradient Boosting", "Test 2")


=== Gradient Boosting - Train ===
Mean Squared Error (MSE): 1344.72
Mean Absolute Error (MAE): 15.61
Sign Accuracy: 0.50
R² Score: -0.03

=== Gradient Boosting - Test 1 ===
Mean Squared Error (MSE): 1417.52
Mean Absolute Error (MAE): 25.63
Sign Accuracy: 0.48
R² Score: -0.02

=== Gradient Boosting - Test 2 ===
Mean Squared Error (MSE): 2010.79
Mean Absolute Error (MAE): 27.70
Sign Accuracy: 0.49
R² Score: -0.00



(np.float64(2010.7925032150767),
 np.float64(27.69574017607925),
 np.float64(0.49382716049382713),
 -0.004111119620333392)

__LSMT__

In [168]:
# Réinitialiser l'index pour éviter les problèmes d'indexation
y_train = y_train.reset_index(drop=True)

# Réinitialisez également X_test1_scaled, si nécessaire, pour correspondre à y_test1
#X_test1_scaled = X_test1_scaled.reset_index(drop=True)
y_test1 = y_test1.reset_index(drop=True)

#X_test2_scaled = X_test2_scaled.reset_index(drop=True)
y_test2 = y_test2.reset_index(drop=True)



In [169]:
import numpy as np

# Définir une fonction pour créer des séquences temporelles
def create_sequences(data, target, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

# Paramètres pour les séquences
sequence_length = 5  # Utiliser 5 observations passées pour prédire la suivante

# Réorganiser vos données
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, sequence_length)
X_test1_seq, y_test1_seq = create_sequences(X_test1_scaled, y_test1, sequence_length)
X_test2_seq, y_test2_seq = create_sequences(X_test2_scaled, y_test2, sequence_length)

print(f"X_train_seq shape: {X_train_seq.shape}")  # [samples, timesteps, features]
print(f"y_train_seq shape: {y_train_seq.shape}")


X_train_seq shape: (6313, 5, 60)
y_train_seq shape: (6313,)


In [173]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

lstm_model = Sequential([
    LSTM(128, input_shape=(sequence_length, X_train_seq.shape[2]), activation='relu', return_sequences=True),
    Dropout(0.3),
    LSTM(64, activation='relu', return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Prédire une seule valeur
])

# Compiler le modèle
lstm_model.compile(optimizer=RMSprop(learning_rate=0.00001), loss='huber')

# Entraînement
lstm_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test1_seq, y_test1_seq),
    epochs=100,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


Epoch 1/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 21.0962 - val_loss: 20.4596 - learning_rate: 1.0000e-05
Epoch 2/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 21.5774 - val_loss: 20.4604 - learning_rate: 1.0000e-05
Epoch 3/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 21.2147 - val_loss: 20.4621 - learning_rate: 1.0000e-05
Epoch 4/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 21.1417 - val_loss: 20.4639 - learning_rate: 1.0000e-05
Epoch 5/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 20.8739 - val_loss: 20.4658 - learning_rate: 1.0000e-05
Epoch 6/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 21.8677 - val_loss: 20.4683 - learning_rate: 1.0000e-05
Epoch 7/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 21.4775 - val_loss: 20.4698 - learning_rate: 5.0000e-06
Epoch 8/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 21.4459 - val_loss: 20.4714 - learning_rate: 5.0000e-06
Epoch 9/100
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 21.

In [174]:
# Prédictions
y_pred_lstm_train = lstm_model.predict(X_train_seq).flatten()
y_pred_lstm_test1 = lstm_model.predict(X_test1_seq).flatten()
y_pred_lstm_test2 = lstm_model.predict(X_test2_seq).flatten()

# Évaluation
evaluate_model(y_train_seq, y_pred_lstm_train, "LSTM", "Train")
evaluate_model(y_test1_seq, y_pred_lstm_test1, "LSTM", "Test 1")
evaluate_model(y_test2_seq, y_pred_lstm_test2, "LSTM", "Test 2")


198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
=== LSTM - Train ===
Mean Squared Error (MSE): 2084.03
Mean Absolute Error (MAE): 22.03
Sign Accuracy: 0.53
R² Score: 0.00

=== LSTM - Test 1 ===
Mean Squared Error (MSE): 765.03
Mean Absolute Error (MAE): 20.95
Sign Accuracy: 0.49
R² Score: -0.01

=== LSTM - Test 2 ===
Mean Squared Error (MSE): 291.55
Mean Absolute Error (MAE): 11.38
Sign Accuracy: 0.42
R² Score: -0.05



(np.float64(291.5493939413414),
 np.float64(11.383961509873059),
 np.float64(0.418848167539267),
 -0.05263716387719297)

In [166]:
print(y_test1.index)

RangeIndex(start=6318, stop=8424, step=1)


In [22]:
import pandas as pd

# Conversion de DELIVERY_START en format numérique
data_cleaned['DELIVERY_START'] = pd.to_datetime(data_cleaned['DELIVERY_START'])
data_cleaned['DELIVERY_START_numeric'] = data_cleaned['DELIVERY_START'].astype(int) // 10**9

# Conversion des colonnes catégorielles en numériques
categorical_cols = data_cleaned.select_dtypes(include=['category']).columns
data_cleaned[categorical_cols] = data_cleaned[categorical_cols].apply(lambda x: x.cat.codes)

# Sélection des colonnes numériques
numeric_data = data_cleaned.select_dtypes(include=['number'])

# Calcul de la corrélation avec la variable cible
correlation_with_spot_id_delta = numeric_data.corr()['spot_id_delta']
print(correlation_with_spot_id_delta)


spot_id_delta                    1.000000
load_forecast                    0.039016
coal_power_available             0.044369
gas_power_available              0.006614
nucelear_power_available         0.039443
wind_power_forecasts_average    -0.003468
solar_power_forecasts_average   -0.027014
wind_power_forecasts_std        -0.015234
solar_power_forecasts_std       -0.009145
predicted_spot_price             0.043844
hour                             0.011612
weekday                         -0.004880
month                           -0.016374
is_weekend                      -0.017934
sin_hour                        -0.011829
cos_hour                         0.003625
sin_weekday                      0.013624
cos_weekday                     -0.002205
sin_month                        0.033920
cos_month                        0.022232
Y_gas_mean_6                     0.006855
Y_gas_std_6                      0.011539
Y_coal_mean_6                    0.044056
Y_coal_std_6                     0

In [179]:
# Sur-échantillonnage avec SMOTE
from imblearn.over_sampling import SMOTE

In [184]:
# Réinitialiser l'index pour éviter les problèmes d'indexation
y_train_class = y_train_class.reset_index(drop=True)

# Réinitialisez également X_test1_scaled, si nécessaire, pour correspondre à y_test1
#X_test1_scaled = X_test1_scaled.reset_index(drop=True)
y_test1_class = y_test1_class.reset_index(drop=True)

#X_test2_scaled = X_test2_scaled.reset_index(drop=True)
y_test2_class = y_test2_class.reset_index(drop=True)

AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

In [188]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Fonction pour transformer y en classification binaire
def transform_target_to_class(target):
    return np.where(target < 0, -1, 1)

# Fonction pour créer des séquences
def create_sequences(data, target, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

# Étape 1 : Transformer y en classification binaire
y_train_class = transform_target_to_class(y_train)
y_test1_class = transform_target_to_class(y_test1)
y_test2_class = transform_target_to_class(y_test2)

# Étape 2 : Réorganiser les données
sequence_length = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_class, sequence_length)
X_test1_seq, y_test1_seq = create_sequences(X_test1_scaled, y_test1_class, sequence_length)
X_test2_seq, y_test2_seq = create_sequences(X_test2_scaled, y_test2_class, sequence_length)

# --- Prétraitement des données ---
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_seq.reshape(X_train_seq.shape[0], -1), y_train_seq)

# Reshape après rééquilibrage
X_train_balanced = X_train_balanced.reshape(-1, X_train_seq.shape[1], X_train_seq.shape[2])

# --- Normalisation des données ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced.reshape(-1, X_train_balanced.shape[2]))
X_train_scaled = X_train_scaled.reshape(-1, X_train_balanced.shape[1], X_train_balanced.shape[2])

X_test1_scaled = scaler.transform(X_test1_seq.reshape(-1, X_test1_seq.shape[2]))
X_test1_scaled = X_test1_scaled.reshape(-1, X_test1_seq.shape[1], X_test1_seq.shape[2])

# --- Définir le modèle LSTM ---
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Classification binaire
])

# --- Compilation du modèle ---
class_weights = {
    0: len(y_train_balanced) / (2 * np.sum(y_train_balanced == 0)),
    1: len(y_train_balanced) / (2 * np.sum(y_train_balanced == 1)),
}

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# --- Définir les callbacks pour l'entraînement ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]

# --- Entraînement du modèle ---
history = model.fit(
    X_train_scaled, y_train_balanced,
    epochs=50,
    batch_size=64,
    validation_data=(X_test1_scaled, y_test1_seq),
    class_weight=class_weights,
    callbacks=callbacks
)

# --- Évaluation du modèle ---
y_pred_proba = model.predict(X_test1_scaled)

# Binariser les résultats
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculer les scores de classification
print("Classification Report:\n", classification_report(y_test1_seq, y_pred))

# AUC-ROC
auc_score = roc_auc_score(y_test1_seq, y_pred_proba)
print("AUC-ROC Score:", auc_score)

# F1-score
f1 = f1_score(y_test1_seq, y_pred)
print("F1 Score:", f1)


IndexError: index 6318 is out of bounds for axis 0 with size 6318

In [25]:
numeric_data = data_cleaned.select_dtypes(include=['float64', 'int64'])
numeric_data = numeric_data.dropna()
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
data_reduced = pca.fit_transform(numeric_data)



[[ 1.20750339e+07  7.72072200e+02  6.64159191e+03 -4.98783186e+03
   1.15047889e+03]
 [ 1.20786340e+07  2.39948738e+02  7.05675668e+03 -4.90085741e+03
   1.27713402e+03]
 [ 1.20822340e+07 -1.41363546e+03  8.06603614e+03 -5.14064820e+03
   1.63616378e+03]
 ...
 [ 1.20606334e+07  1.52943055e+04 -5.27928780e+02 -2.21135732e+03
  -1.47447722e+03]
 [ 1.20642334e+07  1.45972796e+04 -1.22032180e+02 -2.34787930e+03
  -1.32547805e+03]
 [ 1.20678334e+07  1.37725093e+04  3.10788687e+02 -2.59561444e+03
  -1.15511782e+03]]


In [26]:
data_reduced

array([[ 1.20750339e+07,  7.72072200e+02,  6.64159191e+03,
        -4.98783186e+03,  1.15047889e+03],
       [ 1.20786340e+07,  2.39948738e+02,  7.05675668e+03,
        -4.90085741e+03,  1.27713402e+03],
       [ 1.20822340e+07, -1.41363546e+03,  8.06603614e+03,
        -5.14064820e+03,  1.63616378e+03],
       ...,
       [ 1.20606334e+07,  1.52943055e+04, -5.27928780e+02,
        -2.21135732e+03, -1.47447722e+03],
       [ 1.20642334e+07,  1.45972796e+04, -1.22032180e+02,
        -2.34787930e+03, -1.32547805e+03],
       [ 1.20678334e+07,  1.37725093e+04,  3.10788687e+02,
        -2.59561444e+03, -1.15511782e+03]])

# 